In [ ]:
!pip install boto3

In [13]:
import os
import boto3
import json

# Utilitary build functions

In [21]:
def build_ecr_password_stdin(account_id_, region_):
    return f"{account_id_}.dkr.ecr.{region_}.amazonaws.com"

def build_tagged_image(image_name, tag):
    return f"{ecr_image_name}:{tag}"

def build_ecr_url(account_id_, region_, tagged_image_uri_, tag):    
    tagged_image_name=build_tagged_image(image_name, tag)
    password_stdin=build_ecr_password_stdin(account_id_, region_)
    
    return f"{password_stdin}/{tagged_image_name}"

def build_lambda_uri(region_, lambda_arn_):
    uri_host=f"arn:aws:apigateway:{region_}:lambda:path"
    uri_route=f"2015-03-31/functions/{lambda_arn_}/invocations"
    return f"{uri_host}/{uri_route}"

def build_source_arn(region_, account_id_, rest_api_id_):
    return f'arn:aws:execute-api:{region_}:{account_id_}:{rest_api_id_}/*'

def build_api_url(rest_api_id_, region_, endpoint_, stage_):
    host=f"https://{rest_api_id}.execute-api.{region}.amazonaws.com"
    route=f"{stage}/{endpoint}/"
    return f"{host}/{route}"


# Information for communication protocols

In [27]:
# Account
account_id = '060004687794'

# Server
region = "sa-east-1"

# Platform
ecr_image_name = "serverless-example"
platform="linux/amd64"

# API
endpoint = "predict"
method_verb='POST'
stage = "test"

# Ellaborate information
tagged_image_uri=f"{ecr_image_name}:latest"
password_stdin=f"{account_id}.dkr.ecr.{region}.amazonaws.com"

# Specify the role name and trust policy for the Lambda service
trust_policy = {
    'Version': '2012-10-17',
    'Statement': [
        {
            'Effect': 'Allow',
            'Principal': {'Service': 'lambda.amazonaws.com'},
            'Action': 'sts:AssumeRole'
        }
    ]
}

# Rate limits: Harsh since this will be public facing
# Quota: Low daily limits for the same reason
usage_constraints = {
    'rate_limites': {
        'burstLimit': 10,
        'rateLimit': 10.0
    },
    'quota': {
        'limit': 100,
        'period': 'DAY'
    }
}

# Function name (not public facing)
function_name = f'lambda-fn-{ecr_image_name}'


# Clients

In [28]:
# Set up the IAM client
iam_client = boto3.client('iam', region_name=region)

# Set up the Lambda client
lambda_client = boto3.client('lambda', region_name=region)

# Set up the API Gateway client
gateway_client = boto3.client('apigateway', region_name=region)

# Development steps

- IAM role image handling;
- Lambda function creation;
- API Gateway

## IAM Role

In [34]:
# Specify the role name and trust policy for the Lambda service
role_name = 'lambda-exec-role'
policy_arn = 'arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'

# Just need to run it once, otherwise retrieve already existing role
try:
    response = iam_client.get_role(
        RoleName=role_name
    )
except iam_client.exceptions.NoSuchEntityException:
    response = iam_client.create_role(
        RoleName=role_name,
        AssumeRolePolicyDocument=json.dumps(trust_policy),
        Description='Execution role for Lambda function',
    )

# Get the role ARN
role_arn = response['Role']['Arn']

# Attach the AWSLambdaBasicExecutionRole policy to the role
policy_arn = 'arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
response = iam_client.attach_role_policy(
    RoleName=role_name,
    PolicyArn=policy_arn
)

NoCredentialsError: Unable to locate credentials

In [29]:
def try_get_role(i_client, role_name_, trust_policy):
    try:
        return i_client.get_role(
            RoleName=role_name_
        )
    except i_client.exceptions.NoSuchEntityException:
        response = client.create_role(
            RoleName=role_name_,
            AssumeRolePolicyDocument=json.dumps(trust_policy),
            Description='Execution role for Lambda function',
        )
        
def try_attach_role_policy(i_client, role_name_, policy_arn, trust_policy):
    # Just need to run it once, otherwise retrieve already existing role
    response=try_get_role(i_client, role_name_, trust_policy)

    # Get the role ARN
    role_arn = response['Role']['Arn']

    # Attach the AWSLambdaBasicExecutionRole policy to the role
    return i_client.attach_role_policy(RoleName=role_name, PolicyArn=policy_arn)

response = try_attach_role_policy(iam_client, role_name, policy_arn, trust_policy)

In [26]:
def run(command):
    os.system(command)

def login_ecr_docker(account_id, region):
    password_stdin=build_ecr_password_stdin(account_id, region)
    
    get_pwd_command=f"aws ecr get-login-password --region {region}"
    login_command=f"docker login --username AWS --password-stdin {password_stdin}"
    entry_command=f"{get_pwd_command} | {login_command}"
    
    run(entry_command)
    
def create_ecr_image(ecr_image_name_, platform):
    args_1=f"--repository-name {ecr_image_name_}"
    args_2=f"--image-scanning-configuration scanOnPush=true"
    args_3=f"--image-tag-mutability MUTABLE"
    create_args=f"{args_1} {args_2} {args_3}"

    create_comand=f"aws ecr create-repository {create_args}"

    run(create_comand)

def build_docker_image(ecr_image_name, platform):
    build_args=f"-t {ecr_image_name} . --platform={platform}"
    build_command=f"docker build {build_args}"
    
    run(build_command)
    
def tag_docker_image(tagged_image_uri_, routed_url):
    tag_args=f"{tagged_image_uri_} {routed_url}"
    tag_command=f"docker tag {tag_args}"
    
    run(tag_command)
    
def push_docker_image(tagged_image_uri):
    push_command=f"docker push {tagged_image_uri}"

    run(push_command)
    
def pipe_push_image(account_id_, region_, target_platform_, ecr_image_name_, tag):
    password_stdin=build_ecr_password_stdin(account_id_, region_)
    
    # 1. Log in to AWS ECR
    login_ecr_docker(account_id, region)
    
    # 2. Create ECR repo: only needs to be done once
    create_ecr_image(ecr_image_name_, target_platform_)

    # 3. Build Docker image using your local Dockerfile
    build_docker_image(ecr_image_name_, target_platform_)

    # 4. Tag you image
    tagged_image_uri=build_tagged_image(ecr_image_name_, tag)
    tag_docker_image(tagged_image_uri, routed_url)

    # 5. Push your image to ECR
    push_docker_image(tagged_image_uri)

In [ ]:
pipe_push_image(account_id, region, target_platform, ecr_image_name, tag)

# Lambda function 

In [ ]:
def try_get_function(client, function_name, tagged_image_uri):
    success_msg=f"Lambda function {function_name} already exists"
    failure_message=f"Lambda function {function_name} created!"
    
    code_payload={'ImageUri': tagged_image_uri}
    func_description='SKLearn predict Lambda function'
    
    try:
        return client.get_function(FunctionName=function_name)
        print(success_msg)
    except lambda_client.exceptions.ResourceNotFoundException:
        return client.create_function(
            FunctionName=function_name,
            Role=role_arn,
            PackageType='Image',
            Code=code_payload,
            Description=func_description,
            Timeout=10,
            MemorySize=256,
            Publish=True,
        )
        print(failure_message)

In [ ]:
# Retrieve (if already exists) or create a new Lambda function
response = try_get_function(client, function_name, tagged_image_uri)

## API Gateway setup

In [2]:
def has_api(g_client, rest_api_name):
    response = g_client.get_rest_apis()
    create_api_on_gateway = True
    
    for item in response['items']:
        if item['name'] == rest_api_name:
            create_api_on_gateway = False
            
    return create_api_on_gateway

def create_resource(g_client, rest_api_id):
    response = g_client.get_resources(restApiId=rest_api_id)
    root_id = response['items'][0]['id']
    
    response = gateway_client.create_resource(
        restApiId=rest_api_id,
        parentId=root_id,
        pathPart=endpoint,
    )
    
    resource_id = response['id']
    
    return resource_id

def create_rest_method(g_client, resource_id, method_verb):
    g_client.put_method(
        restApiId=rest_api_id,
        resourceId=resource_id,
        httpMethod=method_verb,
        authorizationType='NONE', # WARNING: this will allow public access!
        apiKeyRequired=True,
    )

def create_rest_api(g_client, rest_api_name):
    description='API Gateway that triggers a lambda function'
    reponse=g_client.create_rest_api(name=rest_api_name, description=description) 
    
    rest_api_id = response['id']
    
    return rest_api_id

def get_lambda_arn(g_client, function_name):
    response = g_client.get_function(FunctionName=function_name)
    return response['Configuration']['FunctionArn']

def setup_integration(g_client, lambda_uri, rest_api_id, resource_id, method_verb):
    g_client.put_integration(
        restApiId=rest_api_id,
        resourceId=resource_id,
        httpMethod=method_verb,
        type='AWS_PROXY',
        integrationHttpMethod=method_verb,
        uri=lambda_uri,
    )
    
def create_deployment(g_client, rest_api_id, stage_):
    g_client.create_deployment(restApiId=rest_api_id, stageName=stage_)
    
def create_api_key(g_client, rest_api_name):
    response = g_client.create_api_key(
        name=rest_api_name + '-key',
        description='API key',
        enabled=True,
        generateDistinctId=True
    )
    
    api_key_id = response['id']
    
    return api_key_id

def create_usage_plan(g_client, rest_api_id_, stage_, usage_constraints):
    name='API usage plan'
    description='Harsh rate limits and daily quota for public facing API'
    stages=[
        {
            'apiId': rest_api_id_,
            'stage': stage_,
        },
    ]
    
    # Harsh rate limits since this will be public facing
    constraints={
        'burstLimit': 10,
        'rateLimit': 10.0
    }
    
    # Low daily limits for the same reason
    quota={
        'limit': 100,
        'period': 'DAY'
    }
    
    response = g_client.create_usage_plan(
        name=name,
        description=description,
        apiStages=stages,
        throttle=usage_constraints['rate_limits'],
        quota=usage_constraints['quota']
    )
    
    usage_plan_id = response['id']
    
    return usage_plan_id

def create_usage_plan_key(g_client, usage_plan_id, api_key_id):
    g_client.create_usage_plan_key(
        usagePlanId=usage_plan_id,
        keyId=api_key_id,
        keyType='API_KEY'
    )
    
def add_apigateway_permission(l_client, function_name, source_arn):
    return l_client.add_permission(
        FunctionName=function_name,
        StatementId='apigateway-lambda-invoke-permission',
        Action='lambda:InvokeFunction',
        Principal='apigateway.amazonaws.com',
        SourceArn=source_arn
    )

def deploy_rest_api(\
        g_client, l_client, \
        function_name, rest_api_name, endpoint, method_verb, \
        usage_constraints_, stage_ \
    ):
    # First, lets verify whether we already have an endpoint with this name.
    if not has_api(g_client, rest_api_name):

        # 1. Create REST API
        rest_api_id = create_rest_api(g_client, rest_api_name)

        # 2. Create resource
        resource_id=create_resource(g_client, rest_api_id)

        # 3. Create method
        create_rest_method(g_client, resource_id, method_verb)

        # 4. Get the Lambda function ARN
        lambda_arn = get_lambda_arn(function_name)

        # 5. Set up integration with the Lambda function
        lambda_uri = build_lambda_uri(region, lambda_arn)

        setup_integration(g_client, lambda_uri, rest_api_id, resource_id, method_verb)

        # 6. Deploy API
        create_deployment(g_client, rest_api_id, stage_)

        # 7. Create API key
        api_key_id = create_api_key(g_client, rest_api_name)

        # 8. Create usage plan
        usage_plan_id = create_usage_plan(g_client, rest_api_id_, stage_, usage_constraints_)
        
        # 9. Associate the usage plan with the API key
        create_usage_plan_key(g_client, usage_plan_id, api_key_id)

        # 10. Grant API Gateway permission to invoke the Lambda function
        source_arn = build_source_arn(region, account_id, rest_api_id)

        add_apigateway_permission(l_client, function_name, source_arn)
    
        return {
            'url': build_api_url(rest_api_id, region, endpoint_, stage_),
            'api_key': api_key_id,
            'usage_plan_id': usage_plan_id
        }
    
    else: 
        failure_msg=f"REST API name {rest_api_name} is already under usage!"
        print(failure_msg)
        
        return {}

In [ ]:
# Define the name of the API (not public facing)
rest_api_name = function_name + '-api'

deployment_reponse = deploy_rest_api(\
    gateway_client, lambda_client, \
    function_name, rest_api_name, endpoint, method_verb, \
    usage_constraints, stage \
)


In [14]:
# Prepare the event to pass to the Lambda function
example=[1, 2, 3, 4, 5]

# Tra
payload=json.dumps({"body": example})

# Invoke the Lambda function
response = lambda_client.invoke(
    FunctionName=function_name,
    InvocationType='RequestResponse',
    Payload=payload
)

# Get the response from the Lambda function
result = json.loads(response['Payload'].read())

print(result["body"])

NoCredentialsError: Unable to locate credentials

In [18]:
api_key='Kqrrc4uDk5aFZpH0NLfXW4CvgZphPbrc731nY5Yx'

# The URL by default will follow this pattern:
api_url = build_api_url(rest_api_id_, region_, endpoint_, stage_)
print(api_url)

headers = {
    'Content-type': 'application/json', 
    'x-api-key': api_key,
}  

resp = requests.post(url, headers=headers, json=example)
resp.json()


NameError: name 'rest_api_id' is not defined